In [2]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
vocab_size = 74065
embedding_size = 128

def model_fn(features, labels, mode, params):


    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

   #embedding layer를 선언합니다.
   input_layer = tf.contrib.layers.embed_sequence(
                   features['text'],
                   vocab_size,
                   embedding_size,
                   initializer=params['embedding_initializer']
                   )
   # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
   training = (mode == tf.estimator.ModeKeys.TRAIN)
   # embedding layer에 대한 output에 대해 dropout을 취합니다.
   dropout_emb = tf.layers.dropout(inputs=input_layer,
                                  rate=0.2,
                                  training=training)

   #### CNN 구현체 부분 ####
    conv = tf.layers.conv1d(
           inputs=dropout_emb,
           filters=32,
           kernel_size=3,
           padding='same',
           activation=tf.nn.relu)
  
   pool = tf.reduce_max(input_tensor=conv, axis=1) #max-pooling layer

   hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu) #Fully-connected layer

   #####################


   dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
   logits = tf.layers.dense(inputs=dropout_hidden, units=1)
  
   #prediction 진행 시, None
   if labels is not None:
       labels = tf.reshape(labels, [-1, 1])


    #최종적으로 학습, 평가, 테스트의 단계로 나누어 활용
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )


params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

IndentationError: unindent does not match any outer indentation level (<ipython-input-2-7b55cee8f2d9>, line 14)

In [8]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [9]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
vocab_size = 74065
embedding_size = 128

In [10]:
FILE_PATH = './data/'

In [20]:
train = np.load(FILE_PATH+'nsmc_train_input.npy')
label = np.load(FILE_PATH+'nsmc_train_label.npy')

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [21]:
train_float = train.astype(float)

In [25]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 3          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec
print("Training model...")
model_min5 = word2vec.Word2Vec(train, workers=num_workers, 
           size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling)

2018-08-19 10:38:27,336 : INFO : collecting all words and their counts
2018-08-19 10:38:27,337 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-19 10:38:27,377 : INFO : PROGRESS: at sentence #10000, processed 100000 words, keeping 9595 word types
2018-08-19 10:38:27,421 : INFO : PROGRESS: at sentence #20000, processed 200000 words, keeping 13708 word types
2018-08-19 10:38:27,464 : INFO : PROGRESS: at sentence #30000, processed 300000 words, keeping 16720 word types
2018-08-19 10:38:27,506 : INFO : PROGRESS: at sentence #40000, processed 400000 words, keeping 19238 word types


Training model...


2018-08-19 10:38:27,550 : INFO : PROGRESS: at sentence #50000, processed 500000 words, keeping 21436 word types
2018-08-19 10:38:27,591 : INFO : PROGRESS: at sentence #60000, processed 600000 words, keeping 23311 word types
2018-08-19 10:38:27,638 : INFO : PROGRESS: at sentence #70000, processed 700000 words, keeping 25087 word types
2018-08-19 10:38:27,679 : INFO : PROGRESS: at sentence #80000, processed 800000 words, keeping 26661 word types
2018-08-19 10:38:27,721 : INFO : PROGRESS: at sentence #90000, processed 900000 words, keeping 28197 word types
2018-08-19 10:38:27,760 : INFO : PROGRESS: at sentence #100000, processed 1000000 words, keeping 29673 word types
2018-08-19 10:38:27,799 : INFO : PROGRESS: at sentence #110000, processed 1100000 words, keeping 31036 word types
2018-08-19 10:38:27,840 : INFO : PROGRESS: at sentence #120000, processed 1200000 words, keeping 32370 word types
2018-08-19 10:38:27,881 : INFO : PROGRESS: at sentence #130000, processed 1300000 words, keeping 3

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U11') dtype('<U11') dtype('<U11')

In [6]:
# 파라메터값 지정# 파라메터값  
num_features = 128 # 문자 벡터 차원 수
min_word_count = 6 # 최소 단어 수
num_workers = 4 # 병렬 처리 스레드 수
context = 3 # 문자열 창 크기
downsampling = 1e-3 # 문자 빈도 수 Downsample

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(data, 
                          workers=num_workers, 
                          size=num_features, 
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)

C:\ProgramData\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-08-18 18:03:11,833 : INFO : 'pattern' package not found; tag filters are not available for English
2018-08-18 18:03:11,849 : INFO : collecting all words and their counts
2018-08-18 18:03:11,850 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-18 18:03:11,897 : INFO : PROGRESS: at sentence #10000, processed 116864 words, keeping 12351 word types
2018-08-18 18:03:11,942 : INFO : PROGRESS: at sentence #20000, processed 230625 words, keeping 17363 word types
2018-08-18 18:03:11,982 : INFO : PROGRESS: at sentence #30000, processed 346781 words, keeping 21041 word types
2018-08-18 18:03:12,037 : INFO : PROGRESS: at sentence #40000, processed 463975 words, keeping 24149 word types
2018-08-18 18:03:12,088 : INFO : PROGR

In [7]:
with open('./nsmc/preprocessed_train.pickle', 'rb') as f:
    data = pickle.load(f)

In [46]:
train_string = np.array(train, dtype=str)

In [ ]:
train

In [7]:
input, label = {'text': train}, label

In [8]:
embeded_review = []
NTK = np.zeros((128,), dtype="float32")
idx = 0

for review in data:
    
    if idx % 1000 is 0: print("Complete embedding {0} review".format(idx))
    count = 0
    review_vector = np.zeros((11, 128), dtype="float32")
    len = 0
    for word in review:
        if word in model.wv:
            review_vector[count] = model.wv[word]
        else:
            review_vector[count] = NTK
        count = count + 1
        if(count >10) : break 
    embeded_review.append(review_vector)
    idx = idx + 1
    
embeded_review = np.array(embeded_review)

Complete embedding 0 review
Complete embedding 1000 review
Complete embedding 2000 review
Complete embedding 3000 review
Complete embedding 4000 review
Complete embedding 5000 review
Complete embedding 6000 review
Complete embedding 7000 review
Complete embedding 8000 review
Complete embedding 9000 review
Complete embedding 10000 review
Complete embedding 11000 review
Complete embedding 12000 review
Complete embedding 13000 review
Complete embedding 14000 review
Complete embedding 15000 review
Complete embedding 16000 review
Complete embedding 17000 review
Complete embedding 18000 review
Complete embedding 19000 review
Complete embedding 20000 review
Complete embedding 21000 review
Complete embedding 22000 review
Complete embedding 23000 review
Complete embedding 24000 review
Complete embedding 25000 review
Complete embedding 26000 review
Complete embedding 27000 review
Complete embedding 28000 review
Complete embedding 29000 review
Complete embedding 30000 review
Complete embedding 31

In [54]:
embeded_review = np.array(embeded_review)

In [58]:
embeded_review[1].shape

(11, 128)

In [70]:
review_vector = np.zeros((12, 128), dtype="float32")

In [9]:
labels = label

In [10]:
batch_size = 16
sent_length = 11
vocabulary_size = 128
num_classes = 1


# inputs and labels
sent_inputs = tf.placeholder(shape=[1,sent_length,vocabulary_size],dtype=tf.float32,name='sentence_inputs')
sent_labels = tf.placeholder(shape=[1,num_classes],dtype=tf.float32,name='sentence_labels')



In [63]:
labels

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [11]:
#BATCH_SIZE = 16
#NUM_EPOCHS = 10
#vocab_size = 74065
#embedding_size = 128

#batch_size = 16
#sent_length = 11
#vocabulary_size = 128
#num_classes = 1


conv = tf.layers.conv1d(
           inputs=sent_inputs,
           filters=128,
           kernel_size=3,
           padding='same',
           activation=tf.nn.relu)

pool = tf.reduce_max(input_tensor=conv, axis=1) #max-pooling layer

hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu) 

dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2)

logits = tf.layers.dense(inputs=dropout_hidden, units=1)
logits_sigmoid = tf.sigmoid(input = logits)
pred = tf.cond(logits_sigmoid[0][0]>0.5, lambda: 1, lambda: 0)

global_step = tf.train.get_global_step()
loss = tf.losses.sigmoid_cross_entropy(sent_labels, logits)
train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

In [27]:
logits_sigmoid = tf.sigmoid(logits)

In [64]:
session.run(logits, feed_dict={sent_inputs: embeded_test[556].reshape(1,11,128), sent_labels: test_labels[234].reshape(1,1)})

array([[-0.00912904]], dtype=float32)

In [ ]:
### with open('./test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)


In [14]:
embeded_test = []
NTK = np.zeros((128,), dtype="float32")
idx = 0

for review in test_data:
    
    if idx % 1000 is 0: print("Complete embedding {0} review".format(idx))
    count = 0
    review_vector = np.zeros((11, 128), dtype="float32")
    len = 0
    for word in review:
        if word in model.wv:
            review_vector[count] = model.wv[word]
        else:
            review_vector[count] = NTK
        count = count + 1
        if(count >10) : break 
    embeded_test.append(review_vector)
    idx = idx + 1
    
embeded_test = np.array(embeded_test)



Complete embedding 0 review
Complete embedding 1000 review
Complete embedding 2000 review
Complete embedding 3000 review
Complete embedding 4000 review
Complete embedding 5000 review
Complete embedding 6000 review
Complete embedding 7000 review
Complete embedding 8000 review
Complete embedding 9000 review
Complete embedding 10000 review
Complete embedding 11000 review
Complete embedding 12000 review
Complete embedding 13000 review
Complete embedding 14000 review
Complete embedding 15000 review
Complete embedding 16000 review
Complete embedding 17000 review
Complete embedding 18000 review
Complete embedding 19000 review
Complete embedding 20000 review
Complete embedding 21000 review
Complete embedding 22000 review
Complete embedding 23000 review
Complete embedding 24000 review
Complete embedding 25000 review
Complete embedding 26000 review
Complete embedding 27000 review
Complete embedding 28000 review
Complete embedding 29000 review
Complete embedding 30000 review
Complete embedding 31

In [16]:
session = tf.InteractiveSession()

session.run(pred,feed_dict={sent_inputs: embeded_test[1].reshape(1,11,128), sent_labels: test_labels[1].reshape(1,1)})

FailedPreconditionError: Attempting to use uninitialized value conv1d/bias
	 [[Node: conv1d/bias/read = Identity[T=DT_FLOAT, _class=["loc:@conv1d/bias"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv1d/bias)]]

Caused by op 'conv1d/bias/read', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "C:\P

In [15]:
import pandas as pd

t = pd.read_csv('./data/nsmc/ratings_test.txt', quoting=3, delimiter='\t', header=0)

test_labels = np.array(t['label'])

In [62]:
batch_size = 16
num_steps = 2
#tf.global_variables_initializer().run()
# train_gen = BatchGenerator(batch_size,embeded_review,labels)
# test_gen = BatchGenerator(batch_size,embeded_test,test_labels)

test_interval = 50

# Get accuracy 
def accuracy(labels,preds):
    return np.sum(np.argmax(labels,axis=1)==preds)/labels.shape[0]

print('Initialized\n')


for step in range(num_steps):
    avg_loss = []
    for idx in range(embeded_review.shape[0]):
         
        l,_ = session.run([loss,train_op],feed_dict={sent_inputs: embeded_review[idx].reshape(1,11,128), sent_labels: labels[idx].reshape(1,1)})
        avg_loss.append(l)
        
    print('Train Loss at Epoch %d: %.2f'%(step,np.mean(avg_loss)))
    test_accuracy = []
    #if (step+1)%test_interval==0:        
    #    for te_idx in range(embeded_test.shape[0]):
    #        preds = session.run(pred,feed_dict={sent_inputs: embeded_test[te_idx].reshape(1,11,128), sent_labels: test_labels[te_idx].reshape(1,1)})
    #        test_accuracy.append(accuracy(test_labels,preds))
            
    #    print('Test accuracy at Epoch %d: %.3f'%(step,np.mean(test_accuracy)*100.0))

Initialized

Train Loss at Epoch 0: 0.69
Train Loss at Epoch 1: 0.69


In [23]:
session.run(pred,feed_dict={sent_inputs: embeded_test[327].reshape(1,11,128), sent_labels: test_labels[327].reshape(1,1)})

0

In [18]:
def accuracy(labels,preds):
    return np.sum(labels==preds)/labels.shape[0]

test_accuracy = []
if 0==0:        
    for te_idx in range(embeded_test.shape[0]):
        preds = session.run(prediction,feed_dict={sent_inputs: embeded_test[te_idx].reshape(1,11,128), sent_labels: test_labels[te_idx].reshape(1,1)})
        test_accuracy.append(accuracy(test_labels,preds))

    print('Test accuracy at Epoch %d: %.3f'%(step,np.mean(test_accuracy)*100.0))

Test accuracy at Epoch 2: 2482700.000


In [32]:
session.run(pred,feed_dict={sent_inputs: embeded_test[1].reshape(1,11,128), sent_labels: test_labels[1].reshape(1,1)})

FailedPreconditionError: Attempting to use uninitialized value dense_5/bias
	 [[Node: dense_5/bias/read = Identity[T=DT_FLOAT, _class=["loc:@dense_5/bias"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_5/bias)]]

Caused by op 'dense_5/bias/read', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File 